In [1]:
import pandas as pd

In [2]:
df_chart_tbl = pd.read_csv('D:\Papers\Cyprus_Paper\Data\o31_empty_lab_mean_tbl.csv', index_col=False)
df_mean_chart = pd.read_csv('D:\Papers\Cyprus_Paper\Data\o26_labevents_24hours_mean.csv', index_col=False)
df_data_chart = pd.read_csv('D:\MSc Aegean\Thesis\mimic-iv-1.0\hosp\d_labitems.csv.gz')

In [3]:
"""
Δημιουργώ εκ νέου την στήλη concat
χωρίς κενά ανάμεσα στα στοιχεία.
Θα χρησιμοποιήσω αυτή τη στήλη σαν
index γιατί τα κενά μπορεί να
δημιουργήσουν πρόβλημα.
"""
df_chart_tbl["concat"] = (df_chart_tbl["subject_id"].astype(str)
                          + df_chart_tbl["hadm_id"].astype(str)
                          + df_chart_tbl["charttime"].astype(str))

"""
Αφαιρώ τους χαρακτήρες - και :
από της τιμές της concat για να
αποφύγω πιθανά προβλήματα
"""
df_chart_tbl['concat'] = df_chart_tbl['concat'].str.replace('-','')
df_chart_tbl['concat'] = df_chart_tbl['concat'].str.replace(':','')

In [4]:
# Ορίζω την στήλη concat ως index των γραμμών
df_chart_tbl = df_chart_tbl.set_index('concat')

In [5]:
"""
Δημιουργώ την στήλη concat και στον πίνακα
που περιλαμβάνει τις μετρήσεις για να είναι
το κοινό σημείο ανάμεσα στους δυο πίνακες
"""
df_mean_chart["concat"] = (df_mean_chart["subject_id"].astype(str)
                           + df_mean_chart["hadm_id"].astype(str)
                           + df_mean_chart["charttime"].astype(str))


"""
Αφαιρώ τους χαρακτήρες - και :
από της τιμές της concat για να
αποφύγω πιθανά προβλήματα
"""
df_mean_chart['concat'] = df_mean_chart['concat'].str.replace('-','')
df_mean_chart['concat'] = df_mean_chart['concat'].str.replace(':','')

In [6]:
# Γεμίζω τον πίνακα
for row in df_mean_chart.itertuples():
    con = row.concat
    item = row.itemid
    value = row.Mean_Lab
    df_chart_tbl.at['{}'.format(con), '{}'.format(item)] = value

In [7]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τους κωδικούς των παρατηρήσεων
df_chart_tbl.to_csv('D:\Papers\Cyprus_Paper\Data\o35_fill_lab_mean_itemid_header_tbl.csv', index=False)

In [8]:
df_data_chart = df_data_chart.set_index('itemid')

In [10]:
df_data_chart

,label,fluid,category,loinc_code
itemid,,,,
51905,,Other Body Fluid,Chemistry,NaN
51532,11-Deoxycorticosterone,Blood,Chemistry,NaN
51957,17-Hydroxycorticosteroids,Urine,Chemistry,NaN
51958,"17-Ketosteroids, Urine",Urine,Chemistry,NaN
52068,24 Hr,Blood,Hematology,NaN
...,...,...,...,...
52017,"Zinc, Urine",Urine,Chemistry,NaN
52420,ZZDUMMY,Urine,Hematology,NaN
51771,NaN,Blood,Chemistry,NaN


In [11]:
# Ενοποίηση τιμών σε κελιά
df_data_chart["header"] = ("Mean - "
                           + df_data_chart["label"].astype(str)
                           + " ("
                           + df_data_chart["fluid"].astype(str)
                           + ")")

In [12]:
"""
Ελέγχω τις κεφαλίδες και αν δεν ανήκουν στις
τέσσερις πρώτες τιμές κάνω τις αντικατάσταση.
"""
for col in df_chart_tbl.columns:
    if ((col != 'subject_id') and
        (col != 'hadm_id') and
        (col != 'charttime')):
        temp = (df_data_chart.loc[int(col), 'header'])
        df_chart_tbl.rename(columns={'{}'.format(col): '{}'.format(temp)}, inplace=True)

In [13]:
df_chart_tbl

,subject_id,hadm_id,charttime,Mean - Potassium (Blood),Mean - Creatinine (Blood),Mean - Urea Nitrogen (Blood),Mean - Chloride (Blood),Mean - Sodium (Blood),Mean - Bicarbonate (Blood),Mean - Anion Gap (Blood),...,Mean - Myelocytes (Cerebrospinal Fluid),Mean - Promyelocytes (Pleural),Mean - Non-squamous Epithelial Cells (Urine),Mean - RBC Casts (Urine),Mean - L (Blood),Mean - Young Cells (Blood),Mean - NRBC (Joint Fluid),Mean - H (Blood),Mean - DHEA-Sulfate (Blood),"Mean - Cholesterol, Ascites (Ascites)"
concat,,,,,,,,,,,,,,,,,,,,,
100047332741187621741204,10004733,27411876,2174-12-04,3.55,3.40,44.0,108.5,145.0,25.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741205,10004733,27411876,2174-12-05,3.65,3.45,40.5,106.5,144.0,22.5,18.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741206,10004733,27411876,2174-12-06,3.80,3.80,40.0,106.0,143.0,24.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741207,10004733,27411876,2174-12-07,4.25,3.70,38.5,108.0,144.5,23.0,17.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741208,10004733,27411876,2174-12-08,4.60,3.40,37.0,110.0,145.0,20.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199999872386574521451104,19999987,23865745,2145-11-04,4.10,1.20,22.0,114.0,147.0,26.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199999872386574521451105,19999987,23865745,2145-11-05,3.60,0.90,19.0,110.0,145.0,27.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199999872386574521451106,19999987,23865745,2145-11-06,3.70,1.00,14.0,109.0,143.0,27.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τις ονομασίες των παρατηρήσεων
df_chart_tbl.to_csv('D:\Papers\Cyprus_Paper\Data\o36_fill_mean_lab_label_header_tbl.csv', index=False)